# Doc2Vecの動作確認（一部品詞を落として検証）

## (1) テストデータ／環境準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

## (2) Doc2Vecの動作確認

### (2-1) コーパス生成

コーパス（単語が半角スペースで区切られた文字列）生成時、一部の品詞を落とすようにします。

（＝learning.core.nlang.Nlang クラスの仕様に従います）

In [2]:
import numpy as np

from learning.core.learn.learning_parameter import LearningParameter
from learning.core.datasource import Datasource

_bot_id = 9  # bot_id = 9はセプテーニ
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': 0.5,
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    'params_for_algorithm': {'C': 140},
    'excluded_labels_for_fitting': None
}

learning_parameter = LearningParameter(attr)

In [3]:
_datasource = Datasource(type='csv')
learning_training_messages = _datasource.learning_training_messages(_bot_id)
questions = np.array(learning_training_messages['question'])
answer_ids = np.array(learning_training_messages['answer_id'])

2017/05/17 AM 11:49:52 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/17 AM 11:49:52 ['./fixtures/question_answers/toyotsu_human.csv']


In [4]:
from learning.core.nlang import Nlang

In [5]:
_sentences = np.array(questions)
_separated_sentences = Nlang.batch_split(_sentences)

### (2-2) コーパスにタグ付け

models.doc2vecの仕様に従います。

In [6]:
from gensim import models
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [7]:
def doc_to_sentence(sentences, name):
    words = sentences.split(' ')
    return TaggedDocument(words=words, tags=[name])

def corpus_to_sentences(separated_sentences, answer_ids):
    for idx, (doc, name) in enumerate(zip(separated_sentences, answer_ids)):
        yield doc_to_sentence(doc, name)

### (2-3) 学習処理／モデルのシリアライズ

In [8]:
sentences = corpus_to_sentences(_separated_sentences, answer_ids)

In [9]:
sentence_list = list(sentences)

In [10]:
model = Doc2Vec(size=500, min_count=1, iter=200)

In [11]:
model.build_vocab(sentence_list)

In [12]:
model.train(sentence_list)

13681320

In [13]:
model_path = 'prototype/better_algorithm/doc2vec.model'

model.save(model_path)

In [14]:
'''
    モデル内に保持されているベクトルの数を取得
    （ラベルを回答IDにすると、ラベルの数が戻る。
    　同一回答IDのサンプルのベクトルが、
    　全て上書きされていると考えられる）
'''
len(model.docvecs)

173

In [15]:
model.docvecs

### (2-4) 予測処理

In [16]:
model_path = 'prototype/better_algorithm/doc2vec.model'

def predict(word, model_path):
    '''
        予測処理にかけるコーパスを生成
        （学習セット作成時と同じ関数を使用）
    '''
    corpus = Nlang.split(word).split()

    '''
        コーパスからベクトルを生成し、
        ロードしたモデルから類似ベクトルを検索
    '''
    loaded_model = models.Doc2Vec.load(model_path)
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    return corpus, ret

In [17]:
'''
    マウス破損（正解＝4458）
'''
predict('マウス破損', model_path)

(['マウス', '破損'],
 [(4458, 0.6262366771697998),
  (4530, 0.5883296132087708),
  (4459, 0.5783351063728333),
  (7042, 0.4787851572036743),
  (4499, 0.4761194586753845),
  (4534, 0.46837908029556274),
  (4557, 0.4647749960422516),
  (4520, 0.46033748984336853),
  (7044, 0.4572632610797882),
  (4587, 0.4569905996322632)])

In [18]:
'''
    無線を使用したい（正解＝4516）
'''
predict('無線を使用したい', model_path)

(['無線', '使用', 'する'],
 [(4516, 0.6020066738128662),
  (4493, 0.5783470273017883),
  (4521, 0.569495677947998),
  (4627, 0.5546945929527283),
  (4529, 0.5472049713134766),
  (4542, 0.5420606732368469),
  (4495, 0.5408930778503418),
  (4526, 0.5388065576553345),
  (4594, 0.5382838845252991),
  (4595, 0.5378840565681458)])

In [19]:
'''
    情報システムのアドレス（正解＝7040）
'''
predict('情報システムのアドレス', model_path)

(['情報', 'システム', 'アドレス'],
 [(4444, 0.5556216835975647),
  (4518, 0.554539680480957),
  (4565, 0.5496038198471069),
  (7068, 0.548843502998352),
  (4603, 0.540185809135437),
  (4607, 0.537738561630249),
  (4450, 0.5373344421386719),
  (7084, 0.5280348062515259),
  (4625, 0.5263500809669495),
  (4458, 0.5243576765060425)])

In [20]:
'''
    誤送信防止システムを使いたい（正解＝4432）
'''
predict('誤送信防止システムを使いたい', model_path)

(['誤る', '送信', '防止', 'システム', '使う'],
 [(4458, 0.5284422636032104),
  (4603, 0.49927759170532227),
  (4450, 0.49539750814437866),
  (4530, 0.46389317512512207),
  (4569, 0.4584696292877197),
  (4498, 0.4559299349784851),
  (4626, 0.4530646502971649),
  (7037, 0.45183610916137695),
  (4550, 0.4432029128074646),
  (4494, 0.43968379497528076)])

In [21]:
'''
    携帯からサイボウズを使いたいのですが、どうしたら出来ますか？（正解＝4504）
'''
predict('携帯からサイボウズを使いたいのですが、どうしたら出来ますか？', model_path)

(['携帯', 'サイボウズ', '使う', 'どう', 'する', '出来る'],
 [(4447, 0.6782520413398743),
  (4533, 0.6347713470458984),
  (4595, 0.6024540066719055),
  (4458, 0.5857394933700562),
  (4627, 0.5784133672714233),
  (7084, 0.5756572484970093),
  (4550, 0.5697341561317444),
  (4542, 0.5651775598526001),
  (4552, 0.564193606376648),
  (4494, 0.562157392501831)])